# TAO BYOM Image Classification (TF2) with STM32Cube.AI Developer Cloud

Transfer learning is the process of transfering learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

**Bring Your Own Model (BYOM)** is a Python based package that converts any open source ONNX model to TAO compatible model. All you need is to export any model from a deep learning framework of your choice (e.g. PyTorch) to ONNX and run TAO BYOM converter. More details of installing and running TAO BYOM converter can be found [here](https://github.com/NVIDIA-AI-IOT/tao_byom_examples)


<img align="center" src="TAO-STM32CubeAI.png" width="1080">

## License

This software component is licensed by ST under BSD-3-Clause license,
the "License"; 

You may not use this file except in compliance with the
License. 

You may obtain a copy of the License at: https://opensource.org/licenses/BSD-3-Clause

Copyright (c) 2023 STMicroelectronics. All rights reserved.

Copyright (c) 2023 Nvidia. All rights reserved.

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained mobilenetv2 model and finetune on a sample dataset converted from COCO2014 to perform person detection,
* Prune the finetuned model,
* Retrain the pruned model to recover lost accuracy,
* Export the pruned model as .etlt and then as an onnx model,
* Quantize the model using onnxruntime,
* Run Benchmarking of the quantized onnx model (finetuned, pruned, retrained, and quantized) using STM32Cube.AI Developer Cloud to know the footprints and embeddability of the models.

At the end of this notebook, you will have generated a trained and optimized `classification` model which was imported from outside TAO Toolkit, and that may be deployed via [STM32Cube.AI Developer Cloud](https://stm32ai-cs.st.com/home).

### Table of Contents
This notebook shows an example use case for classification using the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO Launcher](#head-1)
2. [Prepare dataset and pretrained model](#head-2)
    1. [Download, prepare and split the dataset into train/test/val](#head-2-1)
    2. [Check the BYOM model](#head-2-2)
3. [Provide training specification](#head-3)
4. [Finetune the pretrained model using TAO training](#head-4)
5. [Evaluate trained model](#head-5)
    1. [(optional) Export the trained model as onnx format and check the accuracy](#head-5-1)
6. [Prune the trained model](#head-6)
7. [Retrain the pruned model](#head-7)
8. [Testing the finetuned pruned model](#head-8)
    1. [Export the pruned, and retrained model as onnx format](#head-8-1)
    2. [Quantizing the exported onnx model using onnxruntime](#head-8-2)
9. [Benchmarking the optimized model using STM32Cube.AI Developer Cloud for embeddability](#head-9)

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/classification_tf2`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/classification_tf2
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/classification_tf2

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/classification_tf2/byom_person/results_mobilenetv2/
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
os.environ["LOCAL_PROJECT_DIR"] = "/home/user/stm32ai-tao/"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.environ.get("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.environ.get("LOCAL_PROJECT_DIR", os.getcwd()),
    "classification_tf2"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.environ.get("NOTEBOOK_ROOT", os.getcwd()),
    "specs_mobilenetv2"
)

%env SPECS_DIR=/workspace/tao-experiments/classification_tf2/byom_person/specs_mobilenetv2

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from outside to inside of the docker instance.

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    "DockerOptions":{
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. However, **to perform onnx quantization and evaluation using the provided scripts, we recommend to use Python version 3.8.16**. This notebooks is tested for Python version 3.8.16. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8.16

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 <= 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.


In [ ]:
# SKIP this cell IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare datasets and pre-trained model <a class="anchor" id="head-2"></a>

We will be using the modified version of COCO2014 dataset for the tutorial. To find more details please visit this [link](https://pjreddie.com/projects/coco-mirror/). 

#### Download the dataset
To download all the files needed for the dataset in the right location, please uncomment and run the section below.

**NOTE**: If you have already downloaded the dataset files once, you should skip this step.

In [ ]:
!mkdir $LOCAL_DATA_DIR
!wget -O $LOCAL_DATA_DIR/train2014.zip https://pjreddie.com/media/files/train2014.zip
!wget -O $LOCAL_DATA_DIR/val2014.zip https://pjreddie.com/media/files/val2014.zip
!wget -O $LOCAL_DATA_DIR/labels.tgz https://pjreddie.com/media/files/coco/labels.tgz

#### Verify the download.
Checking if the dataset zip files are present in the data directory.

In [ ]:
# Check that file is present
import os
DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
print(DATA_DIR)
if not ( os.path.isfile(os.path.join(DATA_DIR , 'train2014.zip')) and 
        os.path.isfile(os.path.join(DATA_DIR , 'val2014.zip')) and
        os.path.isfile(os.path.join(DATA_DIR , 'labels.tgz')) ):
    print('One or more data files for the dataset are not found.\nPlease download the dataset by running the Download Dataset section!')
else:
    print('Found dataset.')

#### Unpack the files

The downloaded files are in the form of the `zip` and `tgz` format. Running the following code section will unzip and unpack these files.

**NOTE**: If you have already downloaded and unpacked the dataset files once, you can skip this step.

In [ ]:
!unzip $LOCAL_DATA_DIR/train2014.zip -d $LOCAL_DATA_DIR/
!unzip $LOCAL_DATA_DIR/val2014.zip -d $LOCAL_DATA_DIR/
!tar -xzvf $LOCAL_DATA_DIR/labels.tgz -C $LOCAL_DATA_DIR/

Verifying if the files are unpacked as folder.

In [ ]:
!ls $LOCAL_DATA_DIR/

### A. Split the dataset into train/val/test <a class="anchor" id="head-2-1"></a>

For creating the person detection use case we are converting the COCO2014 Dataset into a format where it has only two classes, i.e. `person` and `not_person`. 
In addition to this we are applying an additional filter and removing all the images with person class where the size of the person is too small (covering less than 20% of the image area). 
That is why after preparation instead of 118,287 we have only 84,810 images in total.

In [ ]:
# install pip requirements
!pip3 install tqdm
!pip3 install matplotlib==3.3.3
!pip3 install pandas

Following code section filters the dataset into `person` and `not_person` classes.

In [ ]:
import pandas as pd
import shutil
from pathlib import Path
def filter_coco(area_threshold, labels_dir, input_dir, output_dir):
    """Filter COCO dataset subset filtering person area.

    Args:
      area_threshold: Threshold of fraction of image area below which
      persons are filtered.
      labels_dir: COCO dataset labels directory path.
      input_dir: COCO dataset path.
      output_dir: new dataset output path.
    """
    labels_dpath = Path(labels_dir)
    labels_fpaths = labels_dpath.glob('*.txt')
    input_dpath = Path(input_dir)
    output_dpath = Path(output_dir)
    f = open(input_dpath.stem + '.txt','w+')
    f.write('filename label\n')
    for label_fpath in labels_fpaths:
        img_fname = label_fpath.name.replace('.txt', '.jpg')
        label = 'not_person'
        annotations = pd.read_csv(label_fpath, delimiter=' ', header=None)
        persons = annotations.loc[annotations[0] == 0]
        if persons.shape[0] != 0:
            big_persons = ((persons[3] * persons[4] * 100.0) > area_threshold).sum()
            if big_persons > 0:
                label = 'person'
            else:
                continue
        src_fpath = input_dpath / img_fname
        dst_dpath = output_dpath / label
        dst_dpath.mkdir(parents=True, exist_ok=True)
        shutil.copy(src_fpath, dst_dpath)
        f.write(img_fname + ' ' + label + '\n')
    f.close()
train_labels_path = os.path.join(DATA_DIR, 'labels', 'train2014')
val_labels_path = os.path.join(DATA_DIR, 'labels', 'val2014')
train_images_dir = os.path.join(DATA_DIR, 'train2014')
val_images_dir = os.path.join(DATA_DIR, 'val2014')
result_dir = os.path.join(DATA_DIR, 'person_dataset')
filter_coco(20.0, train_labels_path, train_images_dir, result_dir)
filter_coco(20.0, val_labels_path, val_images_dir, result_dir)

Splitting the dataset into `train`, `val`, `test` portions.

In [ ]:
import os
import glob
import shutil
from random import shuffle
from tqdm import tqdm

DATA_DIR=os.environ["LOCAL_DATA_DIR"]
SOURCE_DIR=os.path.join(DATA_DIR, 'person_dataset')
TARGET_DIR=os.path.join(DATA_DIR,'split')

# removing existing split directory
!rm -rf $TARGET_DIR

# list dir
print(os.walk(SOURCE_DIR))
dir_list = next(os.walk(SOURCE_DIR))[1]
# for each dir, create a new dir in split
for dir_i in tqdm(dir_list):
        newdir_train = os.path.join(TARGET_DIR, 'train', dir_i)
        newdir_val = os.path.join(TARGET_DIR, 'val', dir_i)
        newdir_test = os.path.join(TARGET_DIR, 'test', dir_i)
        
        if not os.path.exists(newdir_train):
                os.makedirs(newdir_train)
        if not os.path.exists(newdir_val):
                os.makedirs(newdir_val)
        if not os.path.exists(newdir_test):
                os.makedirs(newdir_test)

        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(int(len(img_list)*0.7)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'train', dir_i))

        for j in range(int(len(img_list)*0.7), int(len(img_list)*0.8)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'val', dir_i))
                
        for j in range(int(len(img_list)*0.8), len(img_list)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'test', dir_i))
                
print('Done splitting dataset.')

Verifying if the portions are created and all the sub-folders have all the classes.

In [ ]:
!ls $LOCAL_DATA_DIR/split/train
!ls $LOCAL_DATA_DIR/split/val
!ls $LOCAL_DATA_DIR/split/test

### B. Check the BYOM model <a class="anchor" id="head-2-2"></a>

**NOTE**: Before running this step, please run the jupyter notebook [Preparing a TAO BYOM model](../byom_converters/byom_converter_mobilenetv2.ipynb) to prepare the `.tltb` model file as our pretrained model. Place the converted model directory to `$LOCAL_EXPERIMENT_DIR/pretrained_mobilenetv2/mobilenetv2*`.

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_mobilenetv2/

In [ ]:
print("Check that BYOM model was converted into the dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_mobilenetv2/mobilenetv2_128_0_5

## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Training dataset
* Validation dataset
* Pre-trained models
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [ ]:
!cat $LOCAL_SPECS_DIR/spec.yaml

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models

In [ ]:
# logging in the docker session
!docker login nvcr.io

In [ ]:
# delete existing resulting output folder and recreate
!rm -rf $LOCAL_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output
!mkdir -p $LOCAL_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output

In [ ]:
!tao model classification_tf2 train  -e $SPECS_DIR/spec.yaml

### INFO
- To run this training in data parallelism using multiple GPU's, please pass the **number of gpu devices** to be used using `--gpus` parameter. For example for running the training on two gpu devices on parallel the training command will be:
 ```!tao classification_tf2 train -e $SPECS_DIR/spec.yaml --gpus 2```
- The training can be intrupted and then relaunched at any point. To resume from a checkpoint, just relaunch training with the same spec file.

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In this step, we assume that the training is complete and the resulting weights for the model from the some epochs are available. If you would like to run evaluation on an earlier model, you can do so but choosing the resulting weights for that particular epoch, otherwise the scrip below will find the last checkpoint corresponding to the latest finished epoch and run the evaluation on that, by passing it to the configuration parameter `evaluate.model_path`.

In [ ]:
# get the last checkpoints
import os
last_checkpoint = ''
for f in os.listdir(os.path.join(os.environ["LOCAL_EXPERIMENT_DIR"],'byom_person/results_mobilenetv2/output', 'train')):
    if f.startswith('mobilenetv2'):
        last_checkpoint = last_checkpoint if last_checkpoint > f else f
print(f'Last checkpoint: {last_checkpoint}')
%env LAST_CHECKPOINT={last_checkpoint}

In [ ]:
# run the evaluation by overriding the model path for evaluate with the last checkpoint
!tao model classification_tf2 evaluate -e $SPECS_DIR/spec.yaml \
                                 evaluate.checkpoint="$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output/train/$LAST_CHECKPOINT"

### (optional) A. Exporting the trained model to onnx model 
This step is needed if one wants to estimate the footprints of the trained model using STM32Cube.AI.

In [ ]:
!mkdir -p ./results_mobilenetv2/exports
# export the model checkpoint as .onnx file
!tao model classification_tf2 export -e $SPECS_DIR/spec.yaml\
                                 export.checkpoint=$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output/train/$LAST_CHECKPOINT \
                                 export.onnx_file=$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/exports/mobilenetv2.onnx

### (optional) B. Evaluating the onnx model
This is optional sanity check to evaluate the exported trained model to check if the preprocessing in the onnx evaluation and the TAO evaluation are the same.

In [ ]:
!pip install tensorflow==2.9
!pip install onnxruntime==1.14.1
!pip install onnx==1.12.0
!pip install scikit-learn==0.24.2
!pip install matplotlib==3.3.3
!pip install pillow

In [ ]:
# evaluate the onnx model before pruning
import onnx_utils
onnx_utils.evaluate_onnx_model('./results_mobilenetv2/exports/mobilenetv2.onnx', 
                                            os.path.join(os.environ["LOCAL_DATA_DIR"],'split/test/'), save_path='./results_mobilenetv2/mobilenetv2')

## 6. Prune trained models <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion
* Threshold for pruning
* Exclude prediction layer that you don't want pruned (e.g. predictions)

Usually, you just need to adjust `prune.threshold` for accuracy and model size trade off. Higher `threshold` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. `0.5` is just a starting point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

`prune.min_num_filters` is the minimum number of filters to keep per layer after the pruning step. Smaller the value smaller the resulting model. However, this might lower the accuracy. The users can adjust this value depending on their needs.

In [ ]:
!tao model classification_tf2 prune -e $SPECS_DIR/spec.yaml \
                                prune.checkpoint="$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output/train/$LAST_CHECKPOINT"

In [ ]:
print('Pruned model:')
print('------------')
!ls -rlt $LOCAL_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output/prune

## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification

**NOTE**: The path of the pruned model is created based on the `prune.threshold` value automatically. The default path provided in the `spec_retrain.yaml` file is for the value used in this notebook. Make sure to update the path if you change the value of this parameter. 

In [ ]:
!cat $LOCAL_SPECS_DIR/spec_retrain.yaml

In [ ]:
# delete and recreate the directory for the retraining results
!rm -rf $LOCAL_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output_retrain
!mkdir -p $LOCAL_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output_retrain

In [ ]:
!tao model classification_tf2 train  -e $SPECS_DIR/spec_retrain.yaml

## 8. Testing the model! <a class="anchor" id="head-8"></a>

In this step, we assume that the training is complete and the model from the final epoch (`mobilenetv2_xxx.tlt`) is available. If you would like to run evaluation on an earlier model, please edit the spec file at `$SPECS_DIR/spec_retrain.yaml` to point to the intended model.

In [ ]:
# get the last checkpoints
last_checkpoint = ''
for f in os.listdir(os.path.join(os.environ["LOCAL_EXPERIMENT_DIR"],'byom_person/results_mobilenetv2/output_retrain', 'train')):
    if f.startswith('mobilenetv2'):
        last_checkpoint = last_checkpoint if last_checkpoint > f else f
print(f'Last checkpoint: {last_checkpoint}')
%env LAST_CHECKPOINT={last_checkpoint}

In [ ]:
!tao model classification_tf2 evaluate -e $SPECS_DIR/spec_retrain.yaml \
    evaluate.checkpoint="$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output_retrain/train/$LAST_CHECKPOINT"

### A. Exporting the trained model as onnx model <a class="anchor" id="head-8-1"></a>

The following section exports the pruned and retrained model as an onnx model.

In [ ]:
!mkdir -p ./results_mobilenetv2/exports

!tao model classification_tf2 export -e $SPECS_DIR/spec_retrain.yaml\
                                 export.checkpoint=$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/output_retrain/train/$LAST_CHECKPOINT \
                                 export.onnx_file=$USER_EXPERIMENT_DIR/byom_person/results_mobilenetv2/exports/pruned_mobilenetv2.onnx

### A.1. Evaluating the exported onnx model

The following section evaluates the exported onnx model. This should get the same accuracy as the one before converting to onnx.

In [ ]:
!pip install tensorflow==2.9
!pip install onnxruntime==1.14.1
!pip install onnx==1.12.0
!pip install scikit-learn==0.24.2
!pip install matplotlib==3.3.3
!pip install pillow

In [ ]:
# evaluate the onnx model before pruning
import onnx_utils
onnx_utils.evaluate_onnx_model('./results_mobilenetv2/exports/pruned_mobilenetv2.onnx', 
                                            os.path.join(os.environ["LOCAL_DATA_DIR"],'split/test/'), save_path='./results_mobilenetv2/mobilenetv2_pruned')

### B. Quantizing the exported onnx model using onnxruntime <a class="anchor" id="head-8-2"></a>

The following sections converts the exported onnx model to int8 to reduce the footprints and improve the inference time.
This will first require to create a subsample of the data to calibrate the quantization of the model

### B.1. Create a calibration dataset
This should have samples from both classes `person`, and `not_person`.

In [ ]:
# create a subsample dataset for the quantization
import os
import glob
import shutil
from random import shuffle
from tqdm import tqdm

DATA_DIR=os.environ["LOCAL_DATA_DIR"]
SOURCE_DIR=os.path.join(DATA_DIR, 'split/train/')
TARGET_DIR=os.path.join(DATA_DIR,'split/subset_calibration_dataset')

!rm -rf TARGET_DIR

samples_per_class = 200
# list dir
print(os.walk(SOURCE_DIR))
dir_list = next(os.walk(SOURCE_DIR))[1]
# for each dir, create a new dir in split
for dir_i in tqdm(dir_list):
        if not os.path.exists(TARGET_DIR):
                os.makedirs(TARGET_DIR)
                
        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(samples_per_class):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR))                
print('Done creating calibration dataset.')

#### B.2. Quantize the model using QDQ quantization to int8 weights
The following section quantize the float32 onnx model to int8 quantized onnx model.

In [ ]:
# quantize the model
import onnx_utils
onnx_utils.quantize_onnx_model(input_model = './results_mobilenetv2/exports/pruned_mobilenetv2.onnx', 
                              calibration_dataset_path = os.path.join(os.environ["LOCAL_DATA_DIR"],'split','subset_calibration_dataset'))

#### B.3. Evaluate the quantized model

In [ ]:
# evaluate the onnx model before pruning
import onnx_utils
onnx_utils.evaluate_onnx_model('./results_mobilenetv2/exports/pruned_mobilenetv2_QDQ_quant.onnx', 
                                            os.path.join(os.environ["LOCAL_DATA_DIR"],'split/test/'), save_path='./results_mobilenetv2/mobilenetv2_pruned_quantized')

#### B.4. Changing the opset of the onnx model to use them with STM32Cube.AI
The following section will change the opset of the exported onnx models so that they can be used with STM32Cube.AI developer cloud.

In [ ]:
import onnx_utils
onnx_utils.change_opset('./results_mobilenetv2/exports/mobilenetv2.onnx')
onnx_utils.change_opset('./results_mobilenetv2/exports/pruned_mobilenetv2.onnx')
onnx_utils.change_opset('./results_mobilenetv2/exports/pruned_mobilenetv2_QDQ_quant.onnx')

## 9. Benchmarking the optimzed model using STM32Cube.AI Developer Cloud <a class="anchor" id="head-9"></a>

Getting the package for connecting to STM32Cube.AI Developer Cloud.

In [ ]:
!pip install gitdir
!pip install ipywidgets

In [ ]:
!gitdir https://github.com/STMicroelectronics/stm32ai-modelzoo/tree/main/common/stm32ai_dc

import os
import shutil

# Reorganize local folders
if os.path.exists('./stm32ai_dc'):
    shutil.rmtree('./stm32ai_dc')
shutil.move('./common/stm32ai_dc', './stm32ai_dc')
shutil.rmtree('./common')

##### Import, helper and UI functions

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
# import seaborn as sns
import ipywidgets as widgets

sys.path.append(os.path.abspath('stm32ai'))
os.environ['STATS_TYPE'] = 'stm32ai_byom_tao'

# create a directory for outputs for stm32ai developer cloud operations
stm32ai_output_dir = './results_mobilenetv2/stm32ai_outputs'
os.makedirs(stm32ai_output_dir, exist_ok=True)

from stm32ai_dc import (CliLibraryIde, CliLibrarySerie, CliParameters,
                        CloudBackend, Stm32Ai)
from stm32ai_dc.errors import BenchmarkServerError

def analyze_footprints(report=None):
    activations_ram = report.ram_size / 1024
    runtime_ram = report.estimated_library_ram_size / 1024
    total_ram = activations_ram + runtime_ram
    weights_rom = report.rom_size / 1024
    code_rom = report.estimated_library_flash_size / 1024
    total_flash = weights_rom + code_rom
    macc = report.macc / 1e6
    print("[INFO] : STM32Cube.AI model memory footprint")
    print("[INFO] : MACCs : {} (M)".format(macc))
    print("[INFO] : Total Flash : {0:.1f} (KiB)".format(total_flash))
    print("[INFO] :     Flash Weights  : {0:.1f} (KiB)".format(weights_rom))
    print("[INFO] :     Estimated Flash Code : {0:.1f} (KiB)".format(code_rom))
    print("[INFO] : Total RAM : {0:.1f} (KiB)".format(total_ram))
    print("[INFO] :     RAM Activations : {0:.1f} (KiB)".format(activations_ram))
    print("[INFO] :     RAM Runtime : {0:.1f} (KiB)".format(runtime_ram))

def analyze_inference_time(report=None):
    cycles = report.cycles
    inference_time = report.duration_ms
    fps = 1000.0/inference_time
    print("[INFO] : Number of cycles : {} ".format(cycles))
    print("[INFO] : Inference Time : {0:.1f} (ms)".format(inference_time))
    print("[INFO] : FPS : {0:.1f}".format(fps))
    return fps

# UI widget
optimization = ['balanced', 'time', 'ram']
optim_dropdown = widgets.Dropdown(
    options=optimization,
    value=optimization[0],
    description='Optim:',
    disabled=False
)

series_name=["STM32H7", "STM32F7", "STM32F4", "STM32L4", "STM32G4", "STM32F3", "STM32U5", "STM32L5", "STM32F0", "STM32L0", "STM32G0", "STM32C0", "STM32WL"]
series_dropdown = widgets.Dropdown(
    options=series_name,
    value=series_name[0],
    description='Series:',
    disabled=False
)

IDE_name=["gcc", "iar", "keil"]
ide_dropdown = widgets.Dropdown(
    options=IDE_name,
    value=IDE_name[0],
    description='IDE:',
    disabled=False
)

### A. Login to STM32Cube.AI Developer Cloud
Set environment variables with your credentials to acces STM32Cube.AI Developer Cloud.

If you don't have an account yet go to: https://stm32ai-cs.st.com/home and click on sign in to create an account. 

Then set the environment variables below with your credentials.

In [ ]:
import getpass
# Set environment variables with your credentials to access 
# STM32Cube.AI Developer Cloud services
# Fill the username with your login address 
username = 'user.name@example.com'
os.environ['stmai_username'] = username
print('Enter you password')
password = getpass.getpass()
os.environ['stmai_password'] = password

In [ ]:
# Log in STM32Cube.AI Developer Cloud 
try:
    stmai = Stm32Ai(CloudBackend(str(username), str(password)))
    print("Successfully Connected!")
except Exception as e:
    print("Error: please verify your credentials")

### B. Upload the model on STM32Cube.AI Developer Cloud

In [ ]:
# Get the onnx models available locally
model_list = []
for entry in os.listdir('./results_mobilenetv2/exports/'):
  if os.path.isfile(os.path.join('./results_mobilenetv2/exports/', entry)):
    if entry.endswith('.onnx'): model_list.append(entry)
model_sel_dropdown = widgets.Dropdown(
    options=model_list,
    value=model_list[0],
    description='Model:',
    disabled=False
)
display(model_sel_dropdown)

In [ ]:
model_name = model_sel_dropdown.value
model_path = f'./results_mobilenetv2/exports/{model_name}'
model_name = os.path.basename(model_path)
from_model = 'user'

try:
  stmai.upload_model(model_path)
  print(f'Model {model_name} is uploaded !')
except Exception as e:
    print("ERROR: ", e)

### C. Select the STM32Cube.AI optimization setting
- balanced: default compromise between RAM footprint and latency
- time: optimize for latency
- ram: optimize for minimal RAM footprint

In [ ]:
display(optim_dropdown)

### D. Analyze your model memory footprints
stmai.analyze callback parameters:

CLIParameters (options of STM32Cube.AI):

- model: model name corresponding to the file name uploaded.
- optimization: optimization setting "balanced", "time" or "ram".
- allocateInputs: "True" or "False", activations buffer will be also used to handle the input buffers. Optional: True by default.
- allocateOutputs: "True" or "False", activations buffer will be also used to handle the output buffers. Optional: True by default.
- noOnnxOptimizer: "True" or "False", allows to disable the ONNX optimizer pass. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- noOnnxIoTranspose: "True" or "False",  this flag should be used to avoid to add a specific transpose layer during the import of a ONNX model. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- fromModel: to identify the origin model when coming from ST model zoo.

In [ ]:
# Analyze RAM/Flash model memory footprints after optimization by STM32Cube.AI
optimization = optim_dropdown.value
print(f'Anlyzing model : {model_name}, using opimization : {optimization}')
# The runtime library footprint varies slightly depending on the STM32 series
# For an estimation, we use the default series to the STM32F4
try:
  result = stmai.analyze(CliParameters(model=model_path, \
                                       optimization=optimization, \
                                       allocateInputs=True, \
                                       allocateOutputs=True, \
                                       noOnnxIoTranspose=True, \
                                       fromModel=from_model))

  analyze_footprints(report=result)
  # Save the result in outputs folder
  stm32ai_analysis_dir = f'{stm32ai_output_dir}/analysis_report'
  os.makedirs(stm32ai_analysis_dir, exist_ok=True)
  with open(f'./{stm32ai_analysis_dir}/{model_name}_analyze.txt', 'w') as file_analyze:
    file_analyze.write(f'{result}')
except Exception as e:
    print("Error: ", e)

### E. Benchmark your model on a STM32 target
stm32.benchmark callback parameters:

CLIParameters (options of STM32Cube.AI):
- model: model name corresponding to the file name uploaded.
- optimization: optimization setting "balanced", "time" or "ram".
- allocateInputs: "True" or "False", activations buffer will be also used to handle the input buffers. Optional: True by default.
- allocateOutputs: "True" or "False", activations buffer will be also used to handle the output buffers. Optional: True by default.
- noOnnxOptimizer: "True" or "False", allows to disable the ONNX optimizer pass. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- noOnnxIoTranspose: "True" or "False",  this flag should be used to avoid to add a specific transpose layer during the import of a ONNX model. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- fromModel: to identify the origin model when coming from ST model zoo.

Benchmark parameter: board name as string.

In [ ]:
# Get the available board on STM32Cube.AI Developer Cloud
boards = stmai.get_benchmark_boards()
board_names = [boards[i].name for i in range(len(boards))]
print("Available boards:", board_names)

#### Option 1. Benchmark on all available STM32 boards

In [ ]:
# Benchmark the model on STM32Cube.AI Developer Cloud boards
print(f'benchmarking model : {model_name} on all available boards!\nThis may take few minutes!')
fps_array=[]
for board_name in board_names:
  try:
    print(f'STM32 Board: {board_name}')
    result = stmai.benchmark(CliParameters(model=model_name, \
                                           optimization=optimization, \
                                           allocateInputs=True, \
                                           allocateOutputs=True, \
                                           noOnnxIoTranspose=True, \
                                           fromModel=from_model), \
                                           board_name)
    fps = analyze_inference_time(report=result)
    fps_array.append(fps)
    # Save the result in outputs folder
    stm32ai_benchmark_dir = f'{stm32ai_output_dir}/benchmark_report'
    os.makedirs(stm32ai_benchmark_dir, exist_ok=True)
    with open(f'./{stm32ai_benchmark_dir}/{model_name}_{board_name}.txt', 'w') as file_benchmark:
      file_benchmark.write(f'{result}')
  except Exception as e:
    print(f'Not enough memory on {board_name}')
    fps = 0
    fps_array.append(fps)

In [ ]:
# Display the Frame per Second benchmark
sorted_fps = sorted(fps_array, reverse=True)
sorted_boards = [board_names[fps_array.index(i)] for i in sorted_fps]
fig = plt.figure(1, figsize=(15, 8), tight_layout=True)
# colors = sns.color_palette()
colors = ['#4C72B0', '#55A868', '#C44E52', '#8172B2', '#CCB974',
          '#64B5CD', '#B4A7D6', '#AEC7E8', '#FFA07A', '#FFC0CB',
          '#FFFFB3', '#8DD3C7', '#BEBADA', '#FDB462', '#FB8072']

plt.bar(sorted_boards, sorted_fps, color=colors[:len(boards)], width=0.7)
plt.ylabel('FPS', fontsize=15)
plt.yticks(fontsize=12)
plt.xticks(sorted_boards, rotation = 75)
plt.title('STM32 FPS benchmark')
plt.show()

#### Option 2. Benchmark on a selected board

In [ ]:
# Select a board among the available boards
board_dropdown = widgets.Dropdown(
    options=board_names,
    value=board_names[0],
    description='Board:',
    disabled=False,
)
display(board_dropdown)

In [ ]:
board_name = board_dropdown.value
print(f'benchmarking {model_name} on board {board_name}!')
try:
  print(f'STM32 Board: {board_name}')
  result = stmai.benchmark(CliParameters(model=model_name, \
                                         optimization=optimization, \
                                         allocateInputs=True, \
                                         allocateOutputs=True, \
                                         noOnnxIoTranspose=True, \
                                         fromModel=from_model), \
                                         board_name)
  fps = analyze_inference_time(report=result)
  # Save the result in outputs folder
  stm32ai_benchmark_dir = f'{stm32ai_output_dir}/benchmark_report'
  os.makedirs(stm32ai_benchmark_dir, exist_ok=True)
  with open(f'./{stm32ai_benchmark_dir}/{model_name}_{board_name}.txt', 'w') as file_benchmark:
    file_benchmark.write(f'{result}')
except Exception as e:
  print(f'Not enough memory on {board_name}')
  fps = 0

### F. Generate your model optimized C code for STM32

stm32.generate callback parameters:

CLIParameters (options of STM32Cube.AI):

- model: model name corresponding to the file name uploaded.
- optimization: optimization setting "balanced", "time" or "ram".
- allocateInputs: "True" or "False", activations buffer will be also used to handle the input buffers. Optional: True by default.
- allocateOutputs: "True" or "False", activations buffer will be also used to handle the output buffers. Optional: True by default.
- noOnnxOptimizer: "True" or "False", allows to disable the ONNX optimizer pass. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- noOnnxIoTranspose: "True" or "False",  this flag should be used to avoid to add a specific transpose layer during the import of a ONNX model. Optional: "False" by default. Apply only to ONNX file will be ignored otherwise.
- includeLibraryForSerie: include the runtime library for the given STM32 series.
- fromModel: to identify the origin model when coming from ST model zoo.

Choose a target board

In [ ]:
display(series_dropdown)
display(ide_dropdown)

In [ ]:
series = series_dropdown.value
IDE = ide_dropdown.value
print(f'Generating optimized C code of {model_name} model, for {series} series boards!\n')
# Generate model .c/.h code + Lib/Inc on STM32Cube.AI Developer Cloud
stm32ai_code_dir = f'{stm32ai_output_dir}/generated_code'
os.makedirs(stm32ai_code_dir, exist_ok=True)
result = stmai.generate(CliParameters(model=model_name, \
                                      output=stm32ai_code_dir, \
                                      optimization=optimization, \
                                      allocateInputs=True, \
                                      allocateOutputs=True, \
                                      noOnnxIoTranspose=True, \
                                      includeLibraryForSerie=CliLibrarySerie(series), \
                                      includeLibraryForIde=CliLibraryIde(IDE), \
                                      fromModel=from_model))

!ls "{stm32ai_code_dir}"
# print 20 first lines of the report
if os.path.isfile(f'./{stm32ai_code_dir}/network_generate_report.txt'):
  print("\n\n---- code generation report ----\n","*" * 80)
  with open(f'./{stm32ai_code_dir}/network_generate_report.txt', 'r') as f:
    for _ in range(20): print(next(f))

#### You are ready to integrate your model in your STM32 application !

#### (Optional) : Delete your model from your STM32Cube.AI Developer Cloud space

In [ ]:
if stmai.delete_model(model_name):
    print(f'{model_name} deleted from STM32Cube.AI developer Cloud workspace!')